In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image, ImageOps
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib import colormaps
from IPython.display import Image as imgdisp

In [ ]:
# Load all three models
model1 = tf.keras.models.load_model("./model/keras_Model_2.16.1.h5")
model2 = tf.keras.models.load_model("./model/xception_model_v6.h5")
model3 = tf.keras.models.load_model("./model/mobilenetv2_model_v6.h5")

In [ ]:
# Define class names
class_names = ['Basal_Cell_Carcinoma', 'Melanoma', 'Nevus', 'Benign_keratosis', 'No_cancer']

# Define class-specific weights for each model
weights_model1 = np.array([0.6, 0.6, 0.6, 0.6, 0.34])
weights_model2 = np.array([0.2, 0.2, 0.2, 0.2, 0.33])
weights_model3 = np.array([0.2, 0.2, 0.2, 0.2, 0.33])

In [ ]:
# Preprocessing functions
def preprocess_image_model1(img):
    size = (224, 224)
    img = ImageOps.fit(img, size, Image.Resampling.LANCZOS)
    img_array = np.asarray(img)
    img_array = (img_array.astype(np.float32) / 127.5) - 1
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def preprocess_image_model2(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0).astype('float32') / 255
    return img_array

preprocess_image_model3 = preprocess_image_model2

# Combined prediction function
def combined_predict_skin_cancer(img):
    processed_image1 = preprocess_image_model1(img)
    processed_image2 = preprocess_image_model2(img)
    processed_image3 = preprocess_image_model3(img)

    predictions1 = model1.predict(processed_image1)[0]
    predictions2 = model2.predict(processed_image2)[0]
    predictions3 = model3.predict(processed_image3)[0]

    weighted_predictions1 = predictions1 * weights_model1
    weighted_predictions2 = predictions2 * weights_model2
    weighted_predictions3 = predictions3 * weights_model3

    combined_predictions = weighted_predictions1 + weighted_predictions2 + weighted_predictions3
    combined_predictions = combined_predictions / np.sum(combined_predictions)

    predicted_class_index = np.argmax(combined_predictions)
    predicted_class = class_names[predicted_class_index]
    confidence = combined_predictions[predicted_class_index] * 100
    probabilities_percentage = combined_predictions * 100

    return predicted_class, confidence, probabilities_percentage

# Display bar chart for predictions
def display_prediction(preds, class_names):
    fig, ax = plt.subplots(figsize=(6, 3), facecolor='none')
    fig.patch.set_alpha(0)
    
    confidences = preds
    
    ax.barh(class_names, confidences, color=['orange' if conf == max(confidences) else 'lightblue' for conf in confidences])
    
    ax.set_xlim(0, 100)
    ax.set_xlabel('Confidence (%)')
    ax.set_title('Combined Model Prediction Output')
    ax.patch.set_alpha(0)
    
    for i, v in enumerate(confidences):
        ax.text(v + 1, i, f'{v:.1f}%', va='center')
    
    plt.tight_layout()
    plt.show()

# Display the loaded image
def display_image(img):
    fig, ax = plt.subplots(figsize=(4, 4), facecolor='none')
    fig.patch.set_alpha(0)
    ax.imshow(img)
    ax.axis('off')
    ax.set_title('Input Image')
    plt.show()

# Grad-CAM functions
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        model.inputs, 
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

def generate_heatmap(img_array, last_conv_layer_name):
    img_array = preprocess_image_model3(Image.fromarray(img_array))
    model3.layers[-1].activation = None  # Remove last layer's softmax
    return make_gradcam_heatmap(img_array, model3, last_conv_layer_name)

def save_and_display_gradcam(img_array, heatmap, cam_path="./grad_cam/cam.jpg", alpha=0.4):
    # Convert img_array to a PIL image
    img = Image.fromarray(img_array)

    # Resize img_array height to 224 pixels and adjust width to maintain aspect ratio
    width, height = img.size
    new_height = 224
    new_width = int((new_height / height) * width)
    img = img.resize((new_width, new_height))
    img_array = np.array(img)

    # Resize heatmap to match img_array
    heatmap = np.uint8(255 * heatmap)
    heatmap = Image.fromarray(heatmap).resize((new_width, new_height))
    heatmap = np.array(heatmap)

    # Apply colormap to heatmap
    jet = colormaps.get_cmap("jet")
    jet_heatmap = jet(heatmap)[:, :, :3]
    jet_heatmap = np.uint8(jet_heatmap * 255)

    # Superimpose heatmap on the original image
    superimposed_img = jet_heatmap * alpha + img_array * (1 - alpha)
    superimposed_img = np.uint8(superimposed_img)
    
    # Save and display the superimposed image
    superimposed_img = Image.fromarray(superimposed_img)
    superimposed_img.save(cam_path)
    
    display(Image.open(cam_path))
# Load image functions
def load_image_from_url(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    return img

def load_image_from_file(file_content):
    img = Image.open(BytesIO(file_content)).convert("RGB")
    return img

# Prediction handler function
def handle_prediction(img):
    display_image(img)
    predicted_class, confidence, all_probabilities = combined_predict_skin_cancer(img)

    print(f"Predicted class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%")
    print("\nProbabilities for all classes:")
    for class_name, probability in zip(class_names, all_probabilities):
        print(f"{class_name}: {probability:.2f}%")

    display_prediction(all_probabilities, class_names)

    # Only generate and display Grad-CAM if the predicted class is not "No_cancer"
    if predicted_class != "No_cancer":
        img_array = np.array(img)
        heatmap = generate_heatmap(img_array, "block_16_depthwise")
        print("\nGrad-CAM Heatmap:")
        save_and_display_gradcam(img_array, heatmap)
    else:
        print("\nGrad-CAM not displayed for 'No_cancer' prediction.")

# Button handlers
def on_submit_clicked(b):
    with output:
        clear_output()
        if url_input.value:
            img = load_image_from_url(url_input.value)
        elif file_upload.value:
            # Access the first file from the tuple instead of using .values()
            file_content = file_upload.value[0]['content']
            img = load_image_from_file(file_content)
        else:
            print("Please provide a valid URL or upload a file.")
            return
        handle_prediction(img)

def on_exit_clicked(b):
    with output:
        clear_output()
        print("Application closed. You can re-run the cell to start again.")


In [ ]:
# Widgets
url_input = widgets.Text(description="Image URL:", placeholder="Enter the URL of the image")
file_upload = widgets.FileUpload(accept='.png, .jpg, .jpeg', multiple=False)
submit_button = widgets.Button(description="Submit")
exit_button = widgets.Button(description="Exit")
output = widgets.Output()

# Link buttons to functions
submit_button.on_click(on_submit_clicked)
exit_button.on_click(on_exit_clicked)

# Display widgets
display(widgets.VBox([url_input, file_upload, submit_button, exit_button, output]))